* **終了条件** 
  * 生徒の好みが全て一緒であるのにも関わらず、サンドイッチの先頭が違う場合
    * なので生徒の好みが一緒ならその値を、違うなら`#f`を返すような関数を定義
  * 生徒の好みが違うならば、サンドイッチが切り替わる可能性があるので一周させる
    * 一周させて再帰させる

In [16]:
(require racket/function)
(define (same-love? lst)
  (for/fold ([a (car lst)]) ([b (cdr lst)] #:break (not (identity a)))
        (if (equal? a b) a #f)))

(define (one-cycle students foods result)
  (cond 
   [(null? students) (values foods (reverse result))]
   [(equal? (car students) (car foods)) 
    (one-cycle (cdr students) (cdr foods) result)]
   [else 
    (one-cycle (cdr students) foods (cons (car students) result))]))

(define (launch-cycle students foods)
  (if (null? students) 0
  (let ([same-love (same-love? students)])
    (cond 
     [(and same-love (not (equal? same-love (car foods)))) (length students)]
     [else 
      (let-values ([(foods students) (one-cycle students foods (list))])
         (launch-cycle students foods))]))))

(require rackunit)
(check-equal? (same-love? (list 1 1 1)) 1)
(check-equal? (same-love? (list 1 2 1)) #f)

(check-equal? (launch-cycle (list 1 1 0 0) (list 0 1 0 1)) 0)
(check-equal? (launch-cycle (list 1 1 1 0 0 1) (list 1 0 0 0 1 1)) 3)

おそらく、生徒が同じものが好きかどうかを毎回調べるのはオーダー数がかかりすぎるので、1サイクル毎にしたのは正しいのだけれど、1サイクルの関数を書くのにすこし迷った感じがある。たぶん今回はじめて複数の値を返すような関数を定義したのが要因だと思う。

最初の方針としては`struct`を定義したのだけれども、構造体を定義しちゃうとその分余計なオペレーターが増えてコード量が増える。コード量が増えると見通しが悪くなるので良くない。それを意図的にわかりやすくするとなると、関数を定義しないといけない。時間が無限にあればいいけれど、そうもいかない。

となると、そういう場合は基本的に「複数値」を返すような関数を別途用意し、それを受け取るという方針が良くて、そういうコードを書けたのは良かったけど、ただそういう判断がすぐにできるようになると、もう少し素直にかける感じがある。